In [85]:
import numpy as np
import math

ca_data_path = '../evalData/Ca_tot_xsec.txt'
si_data_path = '../evalData/INDEN_Si_tot_xsec.txt'
o_data_path = '../evalData/INDEN_O_tot_xsec.txt'
al_data_path = '../evalData/Al_tot_xsec.txt'
fe_data_path = '../evalData/INDEN_Fe_tot_xsec.txt'
h_data_path = '../evalData/JENDL_H_tot_xsec.txt'
s_endf_data_path = '../evalData/S_tot_xsec.txt'
s_jeff_data_path = '../evalData/JEFF_S_tot_xsec.txt'

min_energy = 1000. #eV

def extract_xsec(file_path, is_sulfur):
    e_arr = []
    xsec_arr = []
    with open(file_path, 'r') as text_file:
        for line in text_file:
            values = line.split()
            if values[0] == '#E,eV':
                continue
            if values[0] == '#END':
                break
            e_arr.append(float(values[0]))
            xsec_arr.append(float(values[1]))
    
    if is_sulfur == True:
        with open(s_jeff_data_path, 'r') as text_file:
            for line in text_file:
                values = line.split()
                if values[0] == '#E,eV':
                    continue
                if values[0] == '#END':
                    break
                if float(values[0]) > 2e7:
                    e_arr.append(float(values[0]))
                    xsec_arr.append(float(values[1]))

    return e_arr, xsec_arr

def fill_xsec_hist(e_arr, xsec_arr, e_bin_edges, xsec_hist):
    xsec_sum = 0
    sum_counter = 0
    bin_counter = 1

    for i in range(0,len(e_arr)):
        if e_arr[i] < min_energy:
            continue
        if e_arr[i] < e_bin_edges[bin_counter]:
            xsec_sum+=xsec_arr[i]
            sum_counter+=1
        if e_arr[i] >= e_bin_edges[bin_counter]:
            if sum_counter == 0:
                xsec_hist[bin_counter-1] = 0
            else:
                xsec_hist[bin_counter-1] = xsec_sum/sum_counter
            xsec_sum = 0
            sum_counter = 0
            bin_counter+=1
            i-=1

    # for i in range(1,len(xsec_arr)):
    #     if xsec_arr[i] == 0:
    #         counter = i
    #         for j in range(i+1,len(xsec_arr)):
    #             if xsec_arr[j] == 0:
    #                 counter+=1
    #                 continue
    #             if xsec_arr[j] != 0:
    #                 break
    #         while (counter-i) >=0:
    #             xsec_arr[counter] = (xsec_arr[i-1] + xsec_arr[counter+1])/2.
    #             counter -= 1
    #     if e_arr[i+1] > 1e6:
    #         break

def fill_trans_hist(xsec_hist, n_value, trans_hist):
    num_bins = len(xsec_hist)
    for i in range(0,num_bins):
        bin_content = math.exp(- n_value * xsec_hist[i])
        trans_hist[i] = bin_content

In [86]:
ca_e_arr, ca_xsec_arr = extract_xsec(ca_data_path, False)
si_e_arr, si_xsec_arr = extract_xsec(si_data_path, False)
o_e_arr, o_xsec_arr = extract_xsec(o_data_path, False)
al_e_arr, al_xsec_arr = extract_xsec(al_data_path, False)
fe_e_arr, fe_xsec_arr = extract_xsec(fe_data_path, False)
h_e_arr, h_xsec_arr = extract_xsec(h_data_path, False)
s_e_arr, s_xsec_arr = extract_xsec(s_endf_data_path, True)

n_ca = 200 * 2.3 * 0.4565 * 0.602214076 / 40.078
n_si = 200 * 2.3 * 0.08724 * 0.602214076 / 28.085
n_o = 200 * 2.3 * 0.35783 * 0.602214076 / 15.999
n_al = 200 * 2.3 * 0.0447 * 0.602214076 / 26.982
n_fe = 200 * 2.3 * 0.04626 * 0.602214076 / 55.845
n_h = 200 * 2.3 * 0.00083 * 0.602214076 / 1.008
n_s = 200 * 2.3 * 0.00664 * 0.602214076 / 32.06

#calculating energy bins
bin_per_decade = 50
min_power = 3
max_power = 9
num_decades = max_power-min_power
num_bins = bin_per_decade * num_decades
bin_edges = []
bin_step = 1./bin_per_decade
for i in range(0,num_bins+1):
    exponent = i * bin_step + min_power
    bin_edges.append(10. ** exponent)

print(num_bins)

300


In [87]:
#cross section hists
ca_xsec_hist = [0] * num_bins
si_xsec_hist = [0] * num_bins
o_xsec_hist = [0] * num_bins
al_xsec_hist = [0] * num_bins
fe_xsec_hist = [0] * num_bins
h_xsec_hist = [0] * num_bins
s_xsec_hist = [0] * num_bins

#filling cross section hists
fill_xsec_hist(ca_e_arr, ca_xsec_arr, bin_edges, ca_xsec_hist)
fill_xsec_hist(si_e_arr, si_xsec_arr, bin_edges, si_xsec_hist)
fill_xsec_hist(o_e_arr, o_xsec_arr, bin_edges, o_xsec_hist)
fill_xsec_hist(al_e_arr, al_xsec_arr, bin_edges, al_xsec_hist)
fill_xsec_hist(fe_e_arr, fe_xsec_arr, bin_edges, fe_xsec_hist)
fill_xsec_hist(h_e_arr, h_xsec_arr, bin_edges, h_xsec_hist)
fill_xsec_hist(s_e_arr, s_xsec_arr, bin_edges, s_xsec_hist)

#transmission hists
ca_trans_hist = [0] * num_bins
si_trans_hist = [0] * num_bins
o_trans_hist = [0] * num_bins
al_trans_hist = [0] * num_bins
fe_trans_hist = [0] * num_bins
h_trans_hist = [0] * num_bins
s_trans_hist = [0] * num_bins

#filling transmission hist
fill_trans_hist(ca_xsec_hist, n_ca, ca_trans_hist)
fill_trans_hist(si_xsec_hist, n_si, si_trans_hist)
fill_trans_hist(o_xsec_hist, n_o, o_trans_hist)
fill_trans_hist(al_xsec_hist, n_al, al_trans_hist)
fill_trans_hist(fe_xsec_hist, n_fe, fe_trans_hist)
fill_trans_hist(h_xsec_hist, n_h, h_trans_hist)
fill_trans_hist(s_xsec_hist, n_s, s_trans_hist)

concrete_trans_hist = [0] * num_bins

for i in range(0, num_bins):
    bin_content = ca_trans_hist[i] * si_trans_hist[i] * o_trans_hist[i] * al_trans_hist[i] * fe_trans_hist[i] * h_trans_hist[i] * s_trans_hist[i]
    concrete_trans_hist[i] = bin_content

print(len(bin_edges))
print(len(concrete_trans_hist))

301
300


In [90]:
bin_edges_low = bin_edges[:-1]
bin_edges_high = bin_edges[1:]

headers = ['Bin Edge Low (eV)', 'Bin Edge High (eV)', 'Total Transmission', 'Ca Transmission', 'Si Transmission', 'O Transmission', 'Al Transmission', 'Fe Transmission', 'H Transmission', 'S Transmission', ]
file_name = "concrete_systematics.csv"
header_string = ",".join(headers)

stacked_array = np.column_stack((bin_edges_low, bin_edges_high, concrete_trans_hist, ca_trans_hist, si_trans_hist, o_trans_hist, al_trans_hist, fe_trans_hist, h_trans_hist, s_trans_hist))
np.savetxt(file_name, stacked_array, delimiter=",", header=header_string)


In [89]:
# import matplotlib.pyplot as plt

# # plt.hist([0.5] * num_bins, bins=bin_edges, weights=concrete_trans_hist, edgecolor='black')
# plt.bar(bin_edges[:-1], concrete_trans_hist, width=np.diff(bin_edges), align='edge', linewidth=1, fill=False)
# plt.xlabel('energy (in eV)')
# plt.ylabel('Transmission')
# plt.title('Transmission hist for concrete')
# plt.xscale('log')
# plt.yscale('log')
# plt.xlim(1e-3,1.5e8)
# plt.show()